# Slackbot Example

SlackBot keeps you in the loop without disturbing your focus. Its personalized, intelligent AI continuously monitors your Slack workspace, alerting you to important conversations and freeing you to concentrate on what’s most important.

SlackBot reads the full history of your (public) Slack workspace and trains a Generative AI model to predict when you need to engage with a conversation. This training process gives the AI a deep understanding of your interests, expertise, and relationships. Using this understanding, SlackBot watches conversations in real-time and notifies you when an important conversation is happening without you. With SlackBot200 you can focus on getting things done without worrying about missing out.

In this notebook, you’ll see you how to build and deploy SlackBot in 15 minutes using only OpenAI’s API’s and open-source Python libraries - Data Science PhD not required.



In [ ]:
!pip install openai kaskada

In [ ]:
from datetime import datetime, timedelta
import openai
import kaskada as k

# Download Slack history from their API
# -> https://slack.com/help/articles/201658943-Export-your-workspace-data
# -> https://api.slack.com/events/message 
# Example Event:
# { 	
#   "type": "message", 	
#   "channel": "C123ABC456", 	
#   "user": "U123ABC456", 	
#   "text": "Hello world", 	
#   "ts": "1355517523.000005" 
# }

# Create a new local session
s = k.session()

# Pull live history into Astra Streaming
# -> Airbyte to Pulsar
# -> https://docs.airbyte.com/integrations/sources/slack
# -> https://docs.airbyte.com/integrations/destinations/pulsar

# Load messages into Kaskada
# For training: message = k.read_parquet(..., entity="user_id")
message = s.read_pulsar(..., entity="user_id")

# Build prompt
messages_by_channel = message.with_key(message.channel)
prompt = k.make_record( 
  # A list of all messages over the past 10 minutes (up to 100)
  recent_messages=messages_by_channel 
    .select("user", "type", "text")
    .last_duration(timedelta(minutes=10), max=100),
  # How many messages have been reacted to in the last 1 minutes
  engage_count=messages_by_channel
    .when(messages_by_channel.get("reactions").is_not_null())
    .count(window=duration(timedelta(minutes=10))),
  # The timestamp of the last message
  time=time_of(messages_by_channel),
)

# We'll train ChatGPT to generate the user ID who will engage next
completion = "\n\n###\n\n" + message.get("user_id")

# Each example will predict who will engage next, given
# the last 10 minutes of conversation history
examples = k.make_record(
  prompt=prompt.lag(1).lookup(message.get("channel")),
  completion=completion,
)
examples = examples.when(examples.get("prompt").is_not_null())

# Split training & validation
from sklearn.model_selection import train_test_split
train, valid = train_test_split(
  examples.to_pandas(),
  test_size=0.2, random_state=42
)
train.to_json("train.jsonl", orient='records', lines=True)
valid.to_json("valid.jsonl", orient='records', lines=True)

# Fine tune a model
!openai api fine_tunes.create -t "train.jsonl" -v "valid.jsonl"

# Apply in real-time
for p in prompt.to_generator(starting=datetime.now()):
  completions = openai.ChatCompletion.create(
    model="ft-2zaA7qi0rxJduWQpdvOvmGn3", 
    messages=[{"role": "user", "content": p}],
    n=5, logprobs=5,
  )
  for completion in completions:
    if sum(completion.logprobs) > 0.05:
      user = completion.completion
      # Use the Slack API to PM the user?
      print(f"Notify {user} of conversation: \n{p}") 

## Legacy Implementation


In [ ]:
%load_ext fenlmagic

In [ ]:
from kaskada.api.session import LocalBuilder
from kaskada import table

session = LocalBuilder().build()

In [ ]:

table.delete_table("Message")

table.create_table(
  # The table's name
  table_name = "Message",
  # The name of the column in the data that contains the time associated with each row
  time_column_name = "ts",
  # The name of the column in the data that contains the entity key associated with each row
  entity_key_column_name = "channel",
  grouping_id = "Channel",
)


In [ ]:

table.load(table_name = "Message", file = "messages.parquet")

In [ ]:
%%fenl
Message

In [ ]:
%%fenl
let message = Message | when(Message.subtype == "message")

let prompt = {
    # TODO: List rather than last
    recent_messages: message | select_fields($input, "user", "text") | last(),

    # # TODO: List support (for reactions field)
    # engage_count: message
    # | when(message.reactions | is_valid())
    # | count(window=sliding(10, minutely())),

    # TODO: the time of day
    time: time_of(message), 
}

let completion = message.user

let examples = {
    prompt: prompt, # TODO: Support lag: | lag(1),
    completion: completion,
}

in examples | when($input.prompt | is_valid())

In [ ]:
# Split training & validation
from sklearn.model_selection import train_test_split
train, valid = train_test_split(
  examples.dataframe,
  test_size=0.2, random_state=42
)
train.to_json("train.jsonl", orient='records', lines=True)
valid.to_json("valid.jsonl", orient='records', lines=True)

# Fine tune a model
!openai api fine_tunes.create -t "train.jsonl" -v "valid.jsonl"